<a href="https://colab.research.google.com/github/nathnascimentto2/FlangeAnnotationTool/blob/master/FlangeAnnotationTool_FasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
#exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 8.9 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=83c58f1cbd6637929889e58f0b595c813a432acf7d8211476023fabb53ffa0b3
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 14 kB/s 
     |████████████████████████████████| 17.3 MB 168 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successf

In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.8")   # please manually install torch 1.8 if Colab changes its default version

1.8.0+cu101 True


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, shutil
from google.colab.patches import cv2_imshow
import subprocess

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
!git clone https://github.com/nathnascimentto2/FlangeAnnotationTool.git

Cloning into 'FlangeAnnotationTool'...
remote: Enumerating objects: 3327, done.
remote: Counting objects: 100% (3327/3327), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 3327 (delta 3264), reused 3315 (delta 3258), pack-reused 0
Receiving objects: 100% (3327/3327), 393.36 KiB | 14.05 MiB/s, done.
Resolving deltas: 100% (3264/3264), done.


In [ ]:
def pascalVOCToCOCO(ann_dir: str, labels: list, output_json: str):

  current_dir = os.getcwd()
  voc2coco = os.path.join(current_dir, 'voc2coco')
  anns_file = os.path.join(current_dir,'annots.txt') 
  labels_file = os.path.join(current_dir,'labels.txt')

  if not os.path.exists(voc2coco):
    !git clone https://github.com/roboflow-ai/voc2coco.git

  with open(labels_file, mode='w') as labelfile:
    for label in labels: labelfile.write(label + '\n')

  with open(anns_file, mode='w') as idfile:
    for id in os.listdir(ann_dir):
      idfile.write('"' + os.path.join(ann_dir, id) + '"\n')

  voc2cocopy = os.path.join(voc2coco, 'voc2coco.py')
  
  if os.path.exists(output_json): os.remove(output_json)

  #subprocess.run(["python", voc2cocopy, "--ann_paths_list", anns_file, "--labels", labels_file, "--output", output_json])
  #print('python {} --ann_paths_list {} --labels {} --output {}'.format(voc2coco, anns_file, labels_file, output_json))
  
  #os.remove(labels_file)
  #os.remove(anns_file)
  #shutil.rmtree(voc2coco)


In [ ]:
ann_dir = '/content/FlangeAnnotationTool/Database/Labels'
labels = ['flange joint']
output_json = '/content/flange_labels.json'
pascalVOCToCOCO(ann_dir, labels, output_json)

In [ ]:
!python '/content/voc2coco/voc2coco.py' --ann_paths_list '/content/annots.txt' --labels '/content/labels.txt' --output '/content/flange_labels.json'

Start converting !
  0% 0/6578 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/content/voc2coco/voc2coco.py", line 150, in <module>
    main()
  File "/content/voc2coco/voc2coco.py", line 145, in main
    extract_num_from_imgid=True
  File "/content/voc2coco/voc2coco.py", line 97, in convert_xmls_to_cocojson
    ann_tree = ET.parse(a_path)
  File "/usr/lib/python3.7/xml/etree/ElementTree.py", line 1197, in parse
    tree.parse(source, parser)
  File "/usr/lib/python3.7/xml/etree/ElementTree.py", line 587, in parse
    source = open(source, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '"/content/FlangeAnnotationTool/Database/Labels/img'


In [ ]:
from detectron2.data.datasets.coco import register_coco_instances
register_coco_instances('raccoon', {}, '/content/raccoon_annts.json', '/content/raccoon_dataset/images')

In [ ]:
DatasetCatalog.get('raccoon')

In [ ]:
dataset_dicts = DatasetCatalog.get('raccoon')
kangaroo_metadata = MetadataCatalog.get('raccoon')

for d in random.sample(dataset_dicts, 8):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=kangaroo_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_C4_3x.yaml'))
cfg.DATASETS.TRAIN = ('raccoon',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_50_C4_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = DatasetCatalog.get('raccoon')
metadata = MetadataCatalog.get('raccoon')
for d in random.sample(dataset_dicts, 50):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

filepath = '/content/img (3290).jpg'
img = cv2.imread(filepath)
cv2_imshow(img)

grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
cv2_imshow(grayscale)

factor = np.ones(grayscale.shape)*-10
contrast = np.add(grayscale, factor) 
cv2_imshow(contrast)

bitmap = contrast.reshape([img.shape[0], img.shape[1]])
bitmap = np.dot((bitmap > 128).astype(float),255)
cv2_imshow(bitmap.astype(np.uint8))